In [1]:
import pandas as pd
import requests
from datetime import datetime
import os

pd.set_option('display.max_columns', None)

# Set your API token
token = "5067f5e2-7e7f-49a7-bf4e-69b069e204c4"  # Replace with your actual token

# Set the headers for the request
headers = {
    "Authorization": f"FPDAPI SubscriberToken={token}",
    "Content-Type": "application/json"
}

# Function to fetch fuel data (as in your original code)
def fetch_fuel_data():
    # Fetch Fuel Types, Brands, Site Details, Site Prices as in your code
    # ...

    # Step 1: Get Fuel Types
    url_fuel_types = "https://fppdirectapi-prod.fuelpricesqld.com.au/Subscriber/GetCountryFuelTypes"
    response_fuel_types = requests.get(url_fuel_types, headers=headers, params={"countryId": "21"})
    df_fuel_types = pd.DataFrame()
    if response_fuel_types.status_code == 200:
        fuel_types_data = response_fuel_types.json()['Fuels']
        df_fuel_types = pd.json_normalize(fuel_types_data)

    # Step 2: Get Brands
    url_brands = "https://fppdirectapi-prod.fuelpricesqld.com.au/Subscriber/GetCountryBrands"
    response_brands = requests.get(url_brands, headers=headers, params={"countryId": "21"})
    df_brands = pd.DataFrame()
    if response_brands.status_code == 200:
        brands_data = response_brands.json()['Brands']
        df_brands = pd.json_normalize(brands_data)

    # Step 3: Get Site Details
    url_site_details = "https://fppdirectapi-prod.fuelpricesqld.com.au/Subscriber/GetFullSiteDetails"
    response_sites = requests.get(url_site_details, headers=headers, params={"countryId": "21", "GeoRegionLevel": "3", "GeoRegionId": "1"})
    df_sites_all = pd.DataFrame()
    if response_sites.status_code == 200:
        sites_data = response_sites.json()['S']  # Assuming the key is 'Sites'
        df_sites_all = pd.json_normalize(sites_data)
        df_sites_all.rename(columns={
            'S': 'Site ID',
            'N': 'Name',
            'A': 'Address',
            'B': 'Brand ID',
            'P': 'Postcode',
            'G1': 'Geographic Region Level 1',
            'G2': 'Geographic Region Level 2',
            'G3': 'Geographic Region Level 3',
            'G4': 'Geographic Region Level 4',
            'G5': 'Geographic Region Level 5',
            'Lat': 'Latitude',
            'Lng': 'Longitude',
            'M': 'Last Modified',
            'GPI': 'Google Place ID'
        }, inplace=True)

    # Step 4: Get Site Prices
    url_site_prices = "https://fppdirectapi-prod.fuelpricesqld.com.au/Price/GetSitesPrices"
    response_prices = requests.get(url_site_prices, headers=headers, params={"countryId": "21", "geoRegionLevel": "3", "geoRegionId": "1"})
    df_fuel_prices = pd.DataFrame()
    if response_prices.status_code == 200:
        fuel_prices_data = response_prices.json()['SitePrices']
        df_fuel_prices = pd.json_normalize(fuel_prices_data)

    # Step 5: Merge DataFrames
    # Merge site details with brands using 'Brand ID'
    merged_df = pd.merge(df_sites_all, df_brands, left_on='Brand ID', right_on='BrandId', how='left')

    # Merge fuel prices with sites using 'Site ID'
    merged_df = pd.merge(merged_df, df_fuel_prices, left_on='Site ID', right_on='SiteId', how='left')

    # Merge fuel types with prices using 'FuelId'
    merged_df = pd.merge(merged_df, df_fuel_types, left_on='FuelId', right_on='FuelId', how='left')

    # Select and rename the desired columns
    final_df = merged_df[[
        'Site ID', 'Address', 'Name_x', 'Postcode', 'Latitude', 
        'Longitude', 'Google Place ID', 'Name_y', 'TransactionDateUtc', 
        'Price', 'Name', 'FuelId'
    ]].rename(columns={
        'Site ID': 'site_id',
        'Address': 'address',
        'Name_x': 'station_name',
        'Postcode': 'postcode',
        'Latitude': 'latitude',
        'Longitude': 'longitude',
        'Google Place ID': 'gp_id',
        'Name_y': 'brand_name',
        'TransactionDateUtc': 'transaction_date_time',
        'Price': 'price',
        'Name': 'fuel_name'
    })


    # Merge data as in your original code
    # final_df = merged_df[...]  # your merged dataframe
    return final_df

# Function to compare and get new/updated data
def get_new_data(new_data, old_data):

    # Ensure postcodes are strings in both DataFrames
    new_data['postcode'] = new_data['postcode'].astype(str)
    old_data['postcode'] = old_data['postcode'].astype(str)

    # Merge new and old data based on site_id and FuelId
    merged = pd.merge(new_data, old_data, how='left', 
                      on=['site_id', 'FuelId'], 
                      suffixes=('_new', '_old'), 
                      indicator=True)
    
    # Find rows where transaction_date_time or price have changed
    diff = merged[
        (merged['_merge'] == 'left_only') |  # New entries
        (merged['transaction_date_time_new'] != merged['transaction_date_time_old']) |  # Date changed
        (merged['price_new'] != merged['price_old'])  # Price changed
    ]
    
    # Keep only the relevant columns (site_id, FuelId, and the changed values)
    diff = diff[['site_id', 'FuelId', 'transaction_date_time_new', 'price_new']].rename(columns={
        'transaction_date_time_new': 'transaction_date_time',
        'price_new': 'price'
    })

    return diff



# File path
file_path = 'fuel_data_oct.csv'

# Step 1: Fetch new data
new_df = fetch_fuel_data()

# Step 2: Check if the file already exists
if os.path.exists(file_path):
    # Load existing data
    old_df = pd.read_csv(file_path)

    # Step 3: Compare and get the new/updated data
    diff_df = get_new_data(new_df, old_df)
    
    if not diff_df.empty:
        # Step 4: Append only new/updated data to CSV
        #diff_df.to_csv(file_path, mode='a', header=False, index=False)
        #print(f"{len(diff_df)} new/updated rows added.")
        print(diff_df.head(5))
    else:
        print("No updates found.")
else:
    # Step 4: Save the new data to CSV if file doesn't exist
    #new_df.to_csv(file_path, index=False)
    print(f"{len(new_df)} rows saved to new file.")

# Step 5: Display new data or message
print(new_df.head())

     site_id  FuelId transaction_date_time   price
10  61401008     2.0   2024-10-03T12:30:00  1979.0
11  61401008     5.0   2024-10-03T12:30:00  2129.0
12  61401008     8.0   2024-10-03T12:30:00  2209.0
13  61401008    12.0   2024-10-03T12:30:00  1959.0
14  61401008    14.0   2024-10-03T12:30:00  1839.0
    site_id            address        station_name postcode   latitude   
0  61290151     61 Burrowes St       Liberty Surat     4417 -27.151627  \
1  61290151     61 Burrowes St       Liberty Surat     4417 -27.151627   
2  61290151     61 Burrowes St       Liberty Surat     4417 -27.151627   
3  61291313  126 Barwon Street  Lowes Mungindi Opt     4497 -28.973667   
4  61291313  126 Barwon Street  Lowes Mungindi Opt     4497 -28.973667   

    longitude                        gp_id brand_name   
0  149.067712  ChIJhbUB8XJVumsR1WCVwJFOMiY    Liberty  \
1  149.067712  ChIJhbUB8XJVumsR1WCVwJFOMiY    Liberty   
2  149.067712  ChIJhbUB8XJVumsR1WCVwJFOMiY    Liberty   
3  148.983999  ChIJWy